
# Initial Setup
Let's begin with imports, as well as environment and figure configuartion.


In [1]:
# cleanup if setup is reinitialized
import os
try:
    os.remove('snorkel.db')
except OSError:
    pass

from snorkel import *
import extractions
import re
import cPickle, sys, matplotlib
import unicodedata
from snorkel import SnorkelSession

session = SnorkelSession()

#sys.path.insert(1, os.path.join(sys.path[0], '..'))
%load_ext autoreload
%autoreload 2
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18,6)


/Users/gmachiraju/py2_kernel/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')



# 1-2) Preprocessing & Candidate (Relation) Extraction Pipeline
This process includes the following steps:
*  [1] Preprocessing our raw data files (includes tagging).
* [2a] Creating "sub"-parsers: Document and Sentence varieties. 
* [2b] Loading our Corpus with a CorpusParser.
* [2c] Candidate Extraction.
* [2d] Relations Generation (directly below; calls steps 1-4).


In [2]:
filename = "articles/"
relations, sent, cand_class, session = extractions.getBiomarkerDiseaseRelations(filename, session)

# dump relations just in case iPython breaks
with open("articles.relations", "w+") as f:
     cPickle.dump(relations, f)
#print relations



corpus made
session not none 1; adding corpus to session
appending doc to corpus 1 Document 28181487
appending doc to corpus 2 Document 28181578
appending doc to corpus 3 Document 28181552
appending doc to corpus 4 Document 28181589
appending doc to corpus 5 Document 28181483
appending doc to corpus 6 Document 28182994
appending doc to corpus 7 Document 5299079
appending doc to corpus 8 Document 5299068
appending doc to corpus 9 Document 28176819
appending doc to corpus 10 Document 28176872
appending doc to corpus 11 Document 28173772
appending doc to corpus 12 Document 5297221
appending doc to corpus 13 Document 28176833
appending doc to corpus 14 Document 28176818
appending doc to corpus 15 Document 28176844
appending doc to corpus 16 Document 28176788
appending doc to corpus 17 Document 28176805
appending doc to corpus 18 Document 28176803
appending doc to corpus 19 Document 28176880
appending doc to corpus 20 Document 28176845
appending doc to corpus 21 Document 28174312
appending 

ValueError: num must be 1 <= num <= 0, not 1

NameError: name 'relations' is not defined


Arbitrary check of POS tagging on our Corpus.


In [6]:
# test = sent[0]
# print test
# print test.words
# print test.pos_tags

# test50 = sent[50]
# print test50
# print test50.words
# print test50.pos_tags

# test100 = sent[100]
# print test100
# print test100.words
# print test100.pos_tags

all_sents = []
for s in sent:
    all_sents.append(s)
    
test = all_sents[0]
print test
print test.words
print test.pos_tags

test50 = all_sents[50]
print test50
print test50.words
print test50.pos_tags

test100 = all_sents[100]
print test100
print test100.words
print test100.pos_tags


Sentence(Document 3560095, 2, u'Improvements were seen in the number of tender joints, the severity of swelling and tenderness, the time of walk 50 feet, the duration of morning stiffness and the circumference of the left knee.')
[u'Improvements', u'were', u'seen', u'in', u'the', u'number', u'of', u'tender', u'joints', u',', u'the', u'severity', u'of', u'swelling', u'and', u'tenderness', u',', u'the', u'time', u'of', u'walk', u'50', u'feet', u',', u'the', u'duration', u'of', u'morning', u'stiffness', u'and', u'the', u'circumference', u'of', u'the', u'left', u'knee', u'.']
[u'NNP', u'VBD', u'VBN', u'IN', u'DT', u'NN', u'IN', u'NN', u'NNS', u',', u'DT', u'NN', u'IN', u'VBG', u'CC', u'NN', u',', u'DT', u'NN', u'IN', u'NN', u'CD', u'NNS', u',', u'DT', u'NN', u'IN', u'NN', u'NN', u'CC', u'DT', u'NN', u'IN', u'DT', u'JJ', u'NN', u'.']
Sentence(Document 10526274, 7, u'Treatment was comprised of VNB, 25 mg/m(2), plus GEM, 1000 mg/m(2), both on Days 1, 8, and 15 every 28 days.')
[u'Treatment', 


# 3a) Internal Evaluation Labeling (Viewer)
Manually generating labels with the viewer. We first define our subclass.


In [7]:
# let's get the candidates
from snorkel.models import CandidateSet
cs = session.query(CandidateSet).filter(CandidateSet.name == 'BD Development Candidates').one()
cs

# Launching the Viewer to inspect Candidates
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(cs[:300], session, annotator_name="BD Viewer User")
sv
#print sv.get_selected()

<IPython.core.display.Javascript object>


# 3b) External Evaluation Label Loading (Ground Truth/"Gold" Labels)

Can be messy since external annotations can be in any format. Let's first load in the Labels from Mind Tagger (tsv format):

In [6]:
import csv

labels = []
with open("gold.tsv") as tsvFile:
    tsvReader = csv.reader(tsvFile, delimiter="\t")
    for line in tsvReader:
        print line
        labels.append(line)
        
formattedTuples = []
for label in labels:
    score = 0
    if label[6] == "true":
        score = 1
    elif label[6] == "false":
        score = -1
    formattedTuples.append((label[5] + "::" + label[3] + "::[" + label[4] + ", " + label[0] + "]::['" + label[1] + "', '" + label[2] + "']", score))

evalIDs = []
evalScores = []
for ft in formattedTuples:
    evalIDs.append(ft[0])
    evalScores.append(ft[1])

['e2_idxs', 'e1_label', 'e2_label', 'sent_id', 'e1_idxs', 'doc_id', 'is_correct']
['[11]', 'MATCHER', 'MATCHER', '44', '[11]', 'pone', 'false']
['[167, 168]', 'MATCHER', 'MATCHER', '4', '[51]', 'PNAS-1998-Belinsky-11891-6', 'false']
['[7]', 'MATCHER', 'MATCHER', '21', '[20]', 'nihms389631', 'false']
['[6, 7]', 'MATCHER', 'MATCHER', '51', '[14]', 'nihms389631', 'true']
['[78, 79]', 'MATCHER', 'MATCHER', '0', '[71]', 'cam40003-1136', 'false']
['[15, 16]', 'MATCHER', 'MATCHER', '6', '[29]', 'nihms143901', 'true']
['[16]', 'MATCHER', 'MATCHER', '9', '[16]', 'pone', 'false']
['[22]', 'MATCHER', 'MATCHER', '21', '[8]', 'PNAS-1998-Belinsky-11891-6', 'false']
['[2]', 'MATCHER', 'MATCHER', '1', '[2]', 'cam40003-1136', 'false']
['[48, 49]', 'MATCHER', 'MATCHER', '17', '[14]', 'pone', 'false']
['[7, 8]', 'MATCHER', 'MATCHER', '2', '[75]', 'nihms356718', 'false']
['[45]', 'MATCHER', 'MATCHER', '17', '[83]', 'pone', 'false']
['[5, 6]', 'MATCHER', 'MATCHER', '9', '[14]', 'nihms143901', 'true']
['[38

In [14]:
from snorkel.loaders import ExternalAnnotationsLoader
loader = ExternalAnnotationsLoader(session, cand_class, 'BD Development Candidates -- Gold','BD Development Labels -- Gold', expand_candidate_set=True)

#utility functions from tutorial
def get_docs_xml(filepath, doc_path=".//document", id_path=".//id/text()"):
    xml = et.fromstring(open(filepath, 'rb').read())
    print xml
    return dict(zip(xml.xpath(id_path), xml.xpath(doc_path)))

def get_CID_unary_mentions(doc_xml, doc, type):
    """
    Get a set of unary disease mentions in argument-dict format,
    for ExternalAnnotationsLoader
    """
    for mesh_id, ms in get_CD_mentions_by_MESHID(doc_xml, doc.sentences)[type].iteritems():
        for m in ms:
            yield {type.lower() : TemporarySpan(parent=m[0], char_start=m[1], char_end=m[2])}

            
import lxml.etree as et
from snorkel.models import CandidateSet, split_stable_id
from snorkel.candidates import TemporarySpan
from collections import defaultdict
import os
import codecs
import csv
import bz2

ROOT = os.getcwd() + "/"
print ROOT

def load_BioC_disease_labels(loader, file_name):
    # Get all the annotated Pubtator documents as XML trees
    doc_xmls = get_docs_xml(ROOT + file_name)
    for doc_id, doc_xml in doc_xmls.iteritems():
    
        # Get the corresponding Document object
        stable_id = "%s::document:0:0" % doc_id
        print stable_id
        doc       = session.query(Document).filter(Document.stable_id == stable_id).first()
        if doc is not None:
        
            # Use custom script + loader to add
            for d in get_CID_unary_mentions(doc_xml, doc, 'Disease'):
                loader.add(d)

load_BioC_disease_labels(loader, 'gold.xml')





# from snorkel.models import Label

# cs = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates -- Gold').one()
# print len(cs)
# print session.query(Label).filter(Label.key == loader.annotation_key).count()

/Users/gmachiraju/Documents/MallickLab/MarkerArk/MarkerArk/
<Element root at 0x115330bd8>



# 4) Training Model via Data Programming
We train our statistical mosel to differentiate between TRUE and FALSE Disease mentions.
This training will be achieved via data programming, enabling us to train a model using only a modest amount of hand-labeled data for validation and testing. We will not use any training labels provided with the skipped usage of training data to simulate a more realistic scenario.


In [5]:
import numpy as np

# set up
train = session.query(CandidateSet).filter(CandidateSet.name == 'BD Training Candidates').one()
#dont have yet:
#dev = session.query(CandidateSet).filter(CandidateSet.name == 'BD Development Candidates').one()



# Generate Features 
We can do so automatically via Snorkel. Recall that the goal is to distinguish TRUE vs FALSE mentions of biomarker-Disease Relations. Hence, we embed our `BD Training Candidates` in a feature space.


In [6]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

# create a feature set
%time F_train = feature_manager.create(session, train, 'Train Features')

# stuff that we have with features...can load interns' features as well (via mindtagger)
#%time F_train2 = feature_manager.load(session, train, 'Train Features')

F_train
F_train.get_candidate(0)
F_train.get_key(0)

#maybe write to another features file: articles

[==========================              ] 63%

Loading sparse Feature matrix...
CPU times: user 1min 19s, sys: 1.21 s, total: 1min 21s
Wall time: 1min 20s


AnnotationKey (DDL_WORD_SEQ_[ALCAM])


# Creating Labeling Functions
Labeling functions are a core tool of data programming. These are heuristic functions that inform the search about the direction to a goal, and therefore aim to classify candidates correctly. Their outputs will be automatically combined and denoised to estimate the probabilities of training labels for the training data.

*Note: We should be creating document-, sentence-, and mention-level labeleing functions.*

In [7]:
import re
#from snorkel.lf_terms import *
from snorkel.lf_helpers import get_doc_candidate_spans
from snorkel.lf_helpers import get_sent_candidate_spans
from snorkel.lf_helpers import get_left_tokens, get_right_tokens

#umls_dict              = load_umls_dictionary()
#chemicals              = load_chemdner_dictionary()
#abbrv2text, text2abbrv = load_specialist_abbreviations()

keyWords = ["associate", "express", "marker", "biomarker", "elevated", "decreased",
            "correlation", "correlates", "found", "diagnose", "variant", "appear",
            "connect", "relate", "exhibit", "indicate", "signify", "show", "demonstrate",
            "reveal", "suggest", "evidence", "elevation", "indication", "diagnosis",
            "variation", "modification", "suggestion", "link", "derivation", "denote",
            "denotation", "demonstration", "magnification", "depression", "boost", "level",
            "advance", "augmentation", "lessening", "enhancement", "expression", "buildup",
            "diminishing", "diminishment", "reduction", "drop", "dwindling", "lowering"]

negationWords = ["not", "nor", "neither"]

# Document-level LFs:
#--------------------
#def LF_undefined_abbreviation(c):
#    '''Candidate is a known abbreviation, but no corresponding full name in document'''
#    doc_spans = get_doc_candidate_spans(c)
#    phrase = c[0].get_span().lower()
#    mentions = set([s.get_span().lower() for s in doc_spans])
#    if len(phrase) > 1 and phrase in abbrv2text and not set(abbrv2text[phrase].keys()).intersection(mentions):
#        return -
    
# Sentence-level LFs:
#---------------------
def LF_contiguous_mentions(c):
    '''Contiguous candidates are likely wrong'''
    neighbor_spans = get_sent_candidate_spans(c)
    start, end = c[0].get_word_start(), c[0].get_word_end()
    for s in neighbor_spans:
        if s.get_word_end() + 1 == start or s.get_word_start() - 1 == end:
            return -1
    return 0

# Mention-level LFs:
#-------------------
def LF_tumors_growths(c):
    phrase = " ".join(c[0].get_attrib_tokens('lemmas'))
    return 1 if re.search("^(\w* ){0,2}(['] )*(tumor|tumour|polyp|pilomatricoma|cyst|lipoma)$", phrase) else 0

def LF_cancer(c):
    '''<TYPE> cancer'''
    phrase = " ".join(c[0].get_attrib_tokens('lemmas'))
    return 1 if re.search("\w* cancer",phrase) else 0

def LF_disease_syndrome(c):
    '''<TYPE> disease or <TYPE> syndrome'''
    phrase = " ".join(c[0].get_attrib_tokens('lemmas'))
    return 1 if re.search("\w* (disease|syndrome)+",phrase) else 0

#def LF_indicators(c):
#    '''Indicator words'''
#    return 1 if " ".join(c[0].get_attrib_tokens()).lower() in indicators else 0

#def LF_common_disease(c):
#    '''Common disease'''
#    return 1 if " ".join(c[0].get_attrib_tokens()).lower() in common_disease else 0

#def LF_common_disease_acronyms(c):
#    '''Common disease acronyms'''
#    return 1 if " ".join(c[0].get_attrib_tokens()) in common_disease_acronyms else 0

def LF_deficiency_of(c):
    '''deficiency of <TYPE>'''
    phrase = " ".join(c[0].get_attrib_tokens()).lower()
    return 1 if phrase.endswith('deficiency') or phrase.startswith('deficiency') or phrase.endswith('dysfunction') else 0

#def LF_positive_indicator(c):
#    flag = False
#    for i in c[0].get_attrib_tokens():
#        if i.lower() in positive_indicator:
#            flag = True
#            break
#    return 1 if flag else 0

def LF_left_positive_argument(c):    
    phrase = " ".join(c[0].get_attrib_tokens('lemmas')).lower()
    pattern = "(\w+ ){1,2}(infection|lesion|neoplasm|attack|defect|anomaly|abnormality|degeneration|carcinoma|lymphoma|tumor|tumour|deficiency|malignancy|hypoplasia|disorder|deafness|weakness|condition|dysfunction|dystrophy)$"
    return 1 if re.search(pattern,phrase) else 0

def LF_right_negative_argument(c):    
    phrase = " ".join(c[0].get_attrib_tokens('lemmas')).lower()
    pattern = "^(history of|mitochondrial|amino acid)( \w+){1,2}"
    return 1 if re.search(pattern, phrase) else 0

def LF_medical_afixes(c):
    pattern = "(\w+(pathy|stasis|trophy|plasia|itis|osis|oma|asis|asia)$|^(hyper|hypo)\w+)"
    phrase = " ".join(c[0].get_attrib_tokens('lemmas')).lower()
    return 1 if re.search(pattern,phrase) else 0

#def LF_adj_diseases(c):
#    return 1 if ' '.join(c[0].get_attrib_tokens()) in adj_diseases else 0


# Dictionary LFs:
#----------------
#def LF_SNOWMED_CT_sign_or_symptom(c):
#    return 1 if c[0].get_span() in umls_dict["snomedct"]["sign_or_symptom"] else 0

#def LF_SNOWMED_CT_disease_or_syndrome(c):
#    return 1 if c[0].get_span() in umls_dict["snomedct"]["disease_or_syndrome"] else 0

#def LF_MESH_disease_or_syndrome(c):
#    return 1 if c[0].get_span() in umls_dict["mesh"]["disease_or_syndrome"] else 0

#def LF_MESH_sign_or_symptom(c):
#    return 1 if c[0].get_span() in umls_dict["mesh"]["sign_or_symptom"] else 0


# Negative LFs:
#--------------
#def LF_organs(c):
#    phrase = " ".join(c[0].get_attrib_tokens()).lower()
#    return -1 if phrase in organs else 0      

#def LF_chemical_name(c):
#    phrase = " ".join(c[0].get_attrib_tokens())
#    return -1 if phrase in chemicals and not phrase.isupper() else 0

#def LF_bodysym(c):
#    phrase = " ".join(c[0].get_attrib_tokens()).lower()
#    return -1 if phrase in bodysym else 0  

def LF_protein_chemical_abbrv(c):
    '''Gene/protein/chemical name'''
    lemma = " ".join(c[0].get_attrib_tokens('lemmas'))
    return -1 if re.search("\d+",lemma) else 0

def LF_base_pair_seq(c): 
    lemma = " ".join(c[0].get_attrib_tokens('lemmas'))
    return -1 if re.search("^[GACT]{2,}$",lemma) else 0

#def LF_too_vague(c):
#    phrase = " ".join(c[0].get_attrib_tokens('lemmas')).lower()
#    phrase_ = " ".join(c[0].get_attrib_tokens()).lower()
#    return -1 if phrase in vague or phrase_ in vague else 0

def LF_neg_surfix(c):
    terms = ['deficiency', 'the', 'the', 'of', 'to', 'a']
    rw = get_right_tokens(c, window=1, attrib='lemmas')
    if len(rw) > 0 and rw[0].lower() in terms:
        return -1
    return 0

#def LF_non_common_disease(c):
#    '''Non common diseases'''
#    return -1 if " ".join(c[0].get_attrib_tokens()).lower() in non_common_disease else 0

#def LF_non_disease_acronyms(c):
#    '''Non common disease acronyms'''
#    return -1 if " ".join(c[0].get_attrib_tokens()) in non_disease_acronyms else 0

def LF_pos_in(c):
    '''Candidates beginning with a preposition or subordinating conjunction'''
    pos_tags = c[0].get_attrib_tokens('pos_tags')
    return -1 if "IN" in pos_tags[0:1] else 0


#def LF_right_window_incomplete(c):
#    return -1 if right_terms.intersection(get_right_tokens(c, window=2, attrib='lemmas')) else 0

#def LF_negative_indicator(c):
#    flag = False
#    for i in c[0].get_attrib_tokens():
#        if i.lower() in negative_indicator:
#            flag = True
#            break
#    return -1 if flag else 0

x = '''
# extra custom
#--------------
def presenceOfNot(m):
    for word in negationWords:
        if (word in m[0].get_right_tokens('lemmas', 20)) and (word in m.pre_window2('lemmas', 20)):
            return True
    return False
# 1
def LF_remove_same_word(m):
    if(m.mention1(attribute='words')[0] == m.mention2(attribute='words')[0]):
        return -1
    
def LF_distance(m):
    print "FIRST"
    print type(m)
    # if 'neuroendocrine' in m.lemmas:
    #     print m.lemmas
    # print m.dep_labels
    distance = abs(m.e2_idxs[0] - m.e1_idxs[0])
    count = 0
    for lemma in m.lemmas:
        if lemma == ',':
            count += 1
    if count > 1 and ',' in m.pre_window1('lemmas', 1):
        print m
        return 0
    if distance == 0:
        return -1
    if distance < 8:
        # print "RETURNING ONE"
        return 0
    else:
        return -1
    
def LF_roman_numeral(m):
    biomarker = (m.mention1(attribute='words')[0])
    unicodedata.normalize('NFKD', biomarker).encode('ascii','ignore')
    if re.match(r'((?<=\s)|(?<=^))(M{1,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{1,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{1,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{1,3}))(?=\s)',
                biomarker):
        print "MATCHED ROMAN"
        print m.mention1(attribute='words')
        return -1

# 4
def LF_marker(m):
    return 1 if ( ('marker' in m[0].get_attrib_tokens('lemmas', 6) or 'biomarker' in m[.post_window1('lemmas', 6)) and (
        'marker' in m.pre_window2('lemmas', 6) or 'biomarker' in m.pre_window2('lemmas', 6)) ) or (('marker' in m.pre_window1('lemmas', 6) or 'biomarker' in m.pre_window1('lemmas', 6)) and (
        'marker' in m.post_window2('lemmas', 6) or 'biomarker' in m.post_window2('lemmas', 6)))  else 0

# 9 (-1 if biomarker is confused with a name of a person)
def LF_People(m):
    return -1 if ('NNP' in m.mention1(attribute='poses')) else 0
# 51
def LF_possible(m):
    return -1 if ('possible' in m.pre_window1('lemmas', 20)) else 0
# 52
def LF_explore(m):
    return -1 if ('explore' in m.pre_window1('lemmas', 20)) else 0
# 53
def LF_key(m):
    # print m.pre_window1('lemmas', 20)
    return -1 if ('abbreviation' in m.pre_window1('lemmas', 20) or (
        'word' in m.pre_window1('lemmas', 20) and 'key' in m.pre_window1('lemmas', 20))) else 0
# 54
def LF_investigate(m):
    return -1 if ('investigate' in m.pre_window1('lemmas', 20)) else 0
# 55
def LF_yetToBeConfirmed(m):
    return -1 if ('yet' in m.post_window1('lemmas', 20) and 'to' in m.post_window1('lemmas', 20) and 'be' in m.post_window1(
        'lemmas', 20) and 'confirmed' in m.post_window1('lemmas', 20)) else 0
# 56
def LF_notAssociated(m):
    return -1 if ('not' in m.post_window1('lemmas', 20) and 'associated' in m.post_window2('lemmas', 20)) else 0
# 56
def LF_notRelated(m):
    return -1 if ('not' in m.post_window1('lemmas', 20) and 'related' in m.post_window2('lemmas', 20)) else 0
# 57
def LF_doesNotShow(m):
    return -1 if (
        'does' in m.post_window1('lemmas', 20) and 'not' in m.post_window1('lemmas', 20) and 'show' in m.post_window2(
            'lemmas', 20)) else 0
# 58
def LF_notLinked(m):
    return -1 if ('not' in m.post_window1('lemmas', 20) and 'linked' in m.post_window2('lemmas', 20)) else 0
# 59
def LF_notCorrelated(m):
    return -1 if ('not' in m.post_window1('lemmas', 20) and 'correlated' in m.post_window2('lemmas', 20)) else 0
# 60
def LF_disprove(m):
    return -1 if ('disprove' in m.post_window1('lemmas', 20)) else 0
# 62
def LF_doesNotSignify(m):
    return -1 if (
        'does' in m.post_window1('lemmas', 20) and 'not' in m.post_window1('lemmas', 20) and 'signify' in m.post_window(
            'lemmas', 20)) else 0
# 63
def LF_doesNotIndicate(m):
    print "SECOND"
    return -1 if (
        'does' in m.post_window1('lemmas', 20) and 'not' in m.post_window1('lemmas', 20) and 'indicate' in m.post_window(
            'lemmas', 20)) else 0
# 64
def LF_doesNotImply(m):
    print "THIRD"
    return -1 if (
        'does' in m.post_window1('lemmas', 20) and 'not' in m.post_window1('lemmas', 20) and 'imply' in m.post_window(
            'lemmas', 20)) else 0
# 65
def LF_studies(m):
    return 1 if (
        'studies' in m.pre_window1('lemmas', 20) and 'have' in m.pre_window1('lemmas', 20) and'reported' in m.pre_window1(
            'lemmas', 20)) else 0
# 66
def LF_studies2(m):
    return 1 if (
        'studies' in m.pre_window1('lemmas', 20) and 'have' in m.pre_window1('lemmas', 20) and 'disclosed' in m.pre_window1(
            'lemmas', 20)) else 0
# 67
def LF_studies3(m):
    return 1 if (
        'studies' in m.pre_window1('lemmas', 20) and 'have' in m.pre_window1('lemmas', 20) and'disclosed' in m.pre_window1('lemmas', 20)) else 0
# 68
def LF_studies4(m):
    return 1 if (
        'studies' in m.pre_window1('lemmas', 20) and 'have' in m.pre_window1('lemmas', 20) and 'expressed' in m.pre_window1(
            'lemmas', 20)) else 0
# 69
def LF_interesting(m):
    return 1 if (
        'is' in m.post_window1('lemmas', 20) and 'an' in m.post_window1('lemmas', 20) and 'interesting' in m.post_window1(
            'lemmas', 20) and 'target' in m.post_window1('lemmas', 20) and 'is' in m.pre_window2('lemmas', 20) and 'an' in
        m.pre_window2('lemmas', 20) and 'interesting' in m.pre_window2('lemmas', 20) and 'target' in m.pre_window2(
            'lemmas', 20)) else 0
# 70
def LF_discussion(m):
    return 1 if (
        'discussion' in m.pre_window1('lemmas', 20)) else 0
# 71
def LF_conclusion(m):
    if ('conclusion' in m.pre_window1('lemmas', 20) or 'conclusion' in m.pre_window2('lemmas', 20)):
        # print "FOUND"
        return 1
    else:
        return 0
# 72
def LF_recently(m):
    return 1 if (
        'recently' in m.pre_window1('lemmas', 20) or 'recently' in m.post_window1('lemmas', 20)) else 0
# 73
def LF_induced(m):
    return 1 if (
        'induced' in m.post_window1('lemmas', 20) and 'induced' in m.pre_window2('lemmas', 20)) else 0
# 74
def LF_treatment(m):
    return 1 if (
        'treatment' in m.pre_window1('lemmas', 20) or 'treatment' in m.post_window1('lemmas', 20)) else 0
# 75
def LF_auxpass(m):
    if not ('auxpass' and 'aux') in (m.post_window1('dep_labels', 20) and m.pre_window2('dep_labels', 20)):
        return -1
    else:
        return 0
# 75
def LF_inbetween(m):
    # with open('diseaseDatabase.pickle', 'rb') as f:
    #     diseaseDictionary = pickle.load(f)
    # with open('diseaseAbbreviationsDatabase.pickle', 'rb') as f:
    #     diseaseAbb = pickle.load(f)
    # with open('markerData.pickle', 'rb') as f:
    #     markerDatabase = pickle.load(f)
    # for marker in markerDatabase:
    #     if(marker in list[m.e1_idxs[0] : m.e2_idxs[0]]):
    #         return -1
    # for disease in diseaseDictionary:
    #     if (disease in list[m.e1_idxs[0]: m.e2_idxs[0]]):
    #         return -1
    # for disease in diseaseAbb:
    #     if (marker in list[m.e1_idxs[0]: m.e2_idxs[0]]):
    #         return -1
    return 0
# 76
def LF_patientsWith(m):
    return 1 if ('patient' in m.post_window2('lemmas', 3)) and ('with' in m.post_window2('lemmas',2)) else 0
# 77
def LF_isaBiomarker(m):
    post_window1_lemmas = m.post_window1('lemmas',20)
    pre_window2_lemmas = m.pre_window2('lemmas',20)
    if ('biomarker' in post_window1_lemmas and 'biomarker' in pre_window2_lemmas) or ('marker' in post_window1_lemmas and 'marker' in pre_window2_lemmas) or ('indicator' in post_window1_lemmas and 'indicator' in pre_window2_lemmas):
        marker_idx_post_window1 = -1
        markers = ['biomarker','marker','indicator']
        for marker in markers:
            try:
                # print post_window1_lemmas
                findMarker = post_window1_lemmas.index(marker)
                if not findMarker == -1:
                    marker_idx_post_window1 = findMarker
                    print marker
            except:
                pass
        if 'cop' in m.post_window1('dep_labels',20):
            try:
                cop_idx_post_window1 = m.post_window1('dep_labels',20).index('cop')
            except:
                pass
            
            print "MarkerIdx:"
            print marker_idx_post_window1
            print "ROOTIdx:"
            try:
                print  m.post_window1('dep_labels',marker_idx_post_window1)
                print  m.post_window1('dep_labels',marker_idx_post_window1).index('ROOT')
            except:
                pass
            print '\n'
            
            return 1 if ('nsubj' in m.mention1(attribute='dep_labels')) and (marker_idx_post_window1-cop_idx_post_window1 < 4)  else 0
    return 0
# 78
def LF_suspect(m):
    return -1 if ('suspect' in m.pre_window1('lemmas', 20) or 'suspect' in m.post_window1('lemmas', 20)) else 0
# 79
def LF_mark(m):
    return -1 if ( 'vmod' in m.post_window1('dep_labels', 20) and 'mark' in m.post_window1('dep_labels', 20) or'vmod' in m.pre_window1('dep_labels', 20) and 'mark' in m.pre_window1('dep_labels', 20)) else 0
'''


# Apply the Labeling Functions


In [8]:
# LFs = LFs_mention + LFs_dicts + LFs_false

#LFs = [LF_investigate, LF_key, LF_distance, LF_auxpass, LF_inbetween,
#       LF_possible, LF_explore, LF_key, LF_investigate, LF_yetToBeConfirmed, LF_notAssociated, LF_notRelated,
#       LF_doesNotShow, LF_notLinked, LF_notCorrelated, LF_disprove, LF_doesNotSignify,
#       LF_doesNotIndicate, LF_doesNotImply, LF_studies, LF_studies2, LF_studies3, LF_studies4, LF_interesting,
#       LF_discussion, LF_conclusion, LF_recently, LF_induced, LF_treatment, LF_isaBiomarker, LF_marker, LF_suspect, LF_mark, LF_People]

LFs = [LF_contiguous_mentions, LF_tumors_growths, LF_cancer, LF_disease_syndrome, LF_deficiency_of, LF_left_positive_argument, LF_right_negative_argument, LF_medical_afixes, LF_protein_chemical_abbrv, LF_base_pair_seq, LF_neg_surfix, LF_pos_in]

# First, we construct a `CandidateLabeler`.
from snorkel.annotations import LabelManager
label_manager = LabelManager()

# Next we run the `CandidateLabeler` to to apply the labeling functions to the training `CandidateSet`. 
# We'll start with some of our labeling functions:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

# statistical summary:
L_train.lf_stats()

[==========================              ] 63%

Loading sparse Label matrix...
CPU times: user 16.8 s, sys: 860 ms, total: 17.7 s
Wall time: 17 s


,conflicts,coverage,j,overlaps
LF_contiguous_mentions,0.0,0.048394,0,0.012758
LF_tumors_growths,0.0,0.000000,1,0.000000
LF_cancer,0.0,0.000000,2,0.000000
LF_disease_syndrome,0.0,0.000000,3,0.000000
LF_deficiency_of,0.0,0.000000,4,0.000000
LF_left_positive_argument,0.0,0.000000,5,0.000000
LF_right_negative_argument,0.0,0.000000,6,0.000000
LF_medical_afixes,0.0,0.002640,7,0.000000
LF_protein_chemical_abbrv,0.0,0.193577,8,0.010559
LF_base_pair_seq,0.0,0.001320,9,0.000440



# Fitting the Generative Model
We estimate the accuracies of the labeling functions without supervision. Specifically, we estimate the parameters of a NaiveBayes generative model.


In [10]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=3000, rate=1e-5)
gen_model.save(session, 'Generative Params')

# We now apply the generative model to the training candidates.
train_marginals = gen_model.marginals(L_train)

Training marginals (!= 0.5):	2273
Features:			12
Begin training for rate=1e-05, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.049023
	Learning epoch = 250	Gradient mag. = 0.051594
	Learning epoch = 500	Gradient mag. = 0.051584
	Learning epoch = 750	Gradient mag. = 0.051575
	Learning epoch = 1000	Gradient mag. = 0.051565
	Learning epoch = 1250	Gradient mag. = 0.051555
	Learning epoch = 1500	Gradient mag. = 0.051546
	Learning epoch = 1750	Gradient mag. = 0.051536
	Learning epoch = 2000	Gradient mag. = 0.051527
	Learning epoch = 2250	Gradient mag. = 0.051517
	Learning epoch = 2500	Gradient mag. = 0.051507
	Learning epoch = 2750	Gradient mag. = 0.051498
Final gradient magnitude for rate=1e-05, mu=1e-06: 0.051



# Training the Discriminative Model
We use the estimated probabilites to train a discriminative model that classifies each `Candidate` as a true or false mention.


In [11]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)
disc_model.w.shape
%time disc_model.save(session, "Discriminative Params")

Training marginals (!= 0.5):	556
Features:			32969
Using gradient descent...
	Learning epoch = 0	Step size = 0.001
	Loss = 385.389832	Gradient magnitude = 258.817501
	Learning epoch = 100	Step size = 0.000904792147114
	Loss = 317.875314	Gradient magnitude = 3.400161
	Learning epoch = 200	Step size = 0.000818648829479
	Loss = 317.389264	Gradient magnitude = 1.613463
	Learning epoch = 300	Step size = 0.000740707032156
	Loss = 317.271242	Gradient magnitude = 0.973397
	Learning epoch = 400	Step size = 0.000670185906007
	Loss = 317.229310	Gradient magnitude = 0.676232
	Learning epoch = 500	Step size = 0.000606378944861
	Loss = 317.210742	Gradient magnitude = 0.516597
	Learning epoch = 600	Step size = 0.000548646907485
	Loss = 317.201289	Gradient magnitude = 0.423235
	Learning epoch = 700	Step size = 0.000496411413431
	Loss = 317.196029	Gradient magnitude = 0.366498
	Learning epoch = 800	Step size = 0.00044914914861
	Loss = 317.192919	Gradient magnitude = 0.331848
	Learning epoch = 900	Step 

# Calibration and Accuracy